In [2]:
import os


In [3]:
os.getcwd()

'/home/ilya/1_Study/Diploma/Input-Output_tables'

In [4]:

# Изменяем текущую директорию на 'data'
os.chdir("data")

# Проверяем, что директория изменилась
print("Текущая директория изменена на:", os.getcwd())

Текущая директория изменена на: /home/ilya/1_Study/Diploma/Input-Output_tables/data


## Try to add 

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Упрощенное описание строк
ROW_NAMES = [
    "Использовано в ценах покупателя",  # 0 (было 73)
    "Оплата труда",                     # 1 (было 74)
    "Другие налоги на производство",    # 2 (было 75)
    "Потребление основного капитала",   # 3 (было 76)
    "Прибыль, смешанный доход",         # 4 (было 77)
    "Доход (Оплата труда + Прибыль)",   # 5 (Новая строка)
    "Импорт"                            # 6 (было 80)
]

def analyze_quadrant3_stability(excel_file):
    """
    Анализирует стабильность показателей 3 квадранта по листам Excel.
    """
    try:
        # Получаем список листов в файле Excel
        excel = pd.ExcelFile(excel_file)
        sheet_names = excel.sheet_names
        
        print(f"Найдено листов в Excel-файле: {len(sheet_names)}")
        
        # Создаем словарь для хранения данных по листам
        data_by_sheet = {}
        
        # Загружаем данные из каждого листа
        for sheet in sheet_names:
            try:
                # Загружаем данные из листа
                df = pd.read_excel(excel_file, sheet_name=sheet)
                
                # Создаем новый DataFrame с тем же количеством столбцов
                new_df = pd.DataFrame(index=range(len(ROW_NAMES)), columns=df.columns)
                
                # Копируем существующие строки
                for i in range(min(5, len(df))):  # Первые 5 строк
                    new_df.iloc[i] = df.iloc[i]
                
                # Вычисляем новую строку: Доход = Оплата труда + Прибыль
                if len(df) >= 5:  # Убеждаемся, что у нас есть доступ к нужным строкам
                    new_df.iloc[5] = df.iloc[1] + df.iloc[4]  # Оплата труда + Прибыль
                
                # Копируем строку Импорт
                if len(df) >= 6:
                    new_df.iloc[6] = df.iloc[5]  # Импорт
                
                # Сохраняем новые данные для листа
                data_by_sheet[sheet] = new_df
                
                print(f"Загружены данные с листа '{sheet}'. Размерность: {new_df.shape}")
            except Exception as e:
                print(f"Ошибка при обработке листа {sheet}: {e}")
        
        if not data_by_sheet:
            print("Не удалось загрузить данные ни с одного листа")
            return None
        
        # Получаем общий список отраслей (колонок)
        all_industries = set()
        for df in data_by_sheet.values():
            all_industries.update(df.columns)
        
        # Создаем список для хранения результатов
        stats_list = []
        
        # Для каждой отрасли и строки собираем данные по всем листам
        for row_idx, row_name in enumerate(ROW_NAMES):
            for industry in all_industries:
                values = []
                
                # Собираем значения для этой пары из всех листов
                for sheet, df in data_by_sheet.items():
                    if industry in df.columns and row_idx < len(df):
                        values.append(df.iloc[row_idx][industry])
                
                # Если есть хотя бы 2 значения, считаем статистику
                if len(values) >= 2:
                    mean_value = np.mean(values)
                    std_value = np.std(values)
                    
                    # Вычисляем коэффициент вариации
                    if mean_value != 0:
                        cv_value = (std_value / abs(mean_value)) * 100
                    else:
                        cv_value = np.nan
                    
                    # Добавляем результаты
                    stats_list.append({
                        'Строка': row_name,
                        'Отрасль': industry,
                        'Среднее': mean_value,
                        'Станд.откл': std_value,
                        'Коэфф.вариации(%)': cv_value,
                        'Кол-во_листов': len(values)
                    })
        
        # Преобразуем в DataFrame
        stats_df = pd.DataFrame(stats_list)
        
        return stats_df
    
    except Exception as e:
        print(f"Ошибка при анализе файла Excel: {e}")
        return None

def identify_stable_industries(stats_df, top_n=3):
    """
    Определяет наиболее стабильные отрасли.
    """
    if stats_df is None or stats_df.empty:
        return None
    
    # Группируем по отраслям и находим средний коэффициент вариации
    industry_stability = stats_df.groupby('Отрасль')['Коэфф.вариации(%)'].mean().reset_index()
    industry_stability.columns = ['Отрасль', 'Средний_коэфф_вариации']
    
    # Сортируем от наиболее стабильных к наименее
    industry_stability = industry_stability.sort_values('Средний_коэфф_вариации')
    
    print(f"\nТоп-{top_n} наиболее стабильных отраслей:")
    print(industry_stability.head(top_n))
    
    return industry_stability

def analyze_stability_for_ces(stats_df):
    """
    Проводит анализ для выбора отраслей, подходящих для CES-функций.
    """
    if stats_df is None or stats_df.empty:
        return None
    
    # Преобразуем DataFrame для упрощения анализа
    pivot_df = pd.pivot_table(
        stats_df, 
        values='Среднее', 
        index='Отрасль',
        columns='Строка'
    )
    
    # Преобразуем DataFrame для коэффициентов вариации
    cv_pivot = pd.pivot_table(
        stats_df, 
        values='Коэфф.вариации(%)', 
        index='Отрасль',
        columns='Строка'
    )
    
    # Список для хранения метрик CES
    ces_results = []
    
    for industry in pivot_df.index:
        # Получаем значения для этой отрасли
        ind_data = pivot_df.loc[industry]
        ind_cv = cv_pivot.loc[industry]
        
        # Базовые компоненты
        labor = ind_data['Оплата труда']
        capital = ind_data['Потребление основного капитала']
        imports = ind_data['Импорт']
        profit = ind_data['Прибыль, смешанный доход']
        taxes = ind_data['Другие налоги на производство']
        income = ind_data['Доход (Оплата труда + Прибыль)']
        
        # Соотношение труд/капитал
        labor_capital_ratio = labor / capital if capital != 0 else np.nan
        
        # Стабильность по коэффициентам вариации
        labor_cv = ind_cv['Оплата труда']
        capital_cv = ind_cv['Потребление основного капитала']
        import_cv = ind_cv['Импорт']
        income_cv = ind_cv['Доход (Оплата труда + Прибыль)']
        
        # Средняя стабильность соотношения труд/капитал
        labor_capital_stability = (labor_cv + capital_cv) / 2
        
        # Добавленная стоимость
        value_added = labor + taxes + capital + profit
        
        # Доли компонентов
        labor_share = labor / value_added * 100 if value_added > 0 else 0
        capital_share = capital / value_added * 100 if value_added > 0 else 0
        profit_share = profit / value_added * 100 if value_added > 0 else 0
        income_share = income / value_added * 100 if value_added > 0 else 0
        
        # Комплексный индекс стабильности (меньше = лучше)
        ces_index = (
            0.4 * labor_capital_stability +  # 40% вес
            0.3 * import_cv +                # 30% вес
            0.3 * (imports * 100)            # 30% вес
        )
        
        # Добавляем результаты
        ces_results.append({
            'Отрасль': industry,
            'Труд/Капитал': labor_capital_ratio,
            'Стабильность_ТК': labor_capital_stability,
            'Доля_импорта': imports * 100,
            'Стабильность_импорта': import_cv,
            'Доля_труда': labor_share,
            'Доля_капитала': capital_share,
            'Доля_прибыли': profit_share,
            'Доля_дохода': income_share,
            'Стабильность_дохода': income_cv,
            'CES_индекс': ces_index
        })
    
    # Создаем DataFrame и сортируем по индексу стабильности
    ces_df = pd.DataFrame(ces_results)
    if not ces_df.empty:
        ces_df = ces_df.sort_values('CES_индекс')
    
    return ces_df

def create_stability_heatmap(stats_df, output_file='stability_heatmap.png'):
    """
    Создает тепловую карту коэффициентов вариации.
    """
    if stats_df is None or stats_df.empty:
        print("Нет данных для создания тепловой карты")
        return
    
    # Создаем сводную таблицу для тепловой карты
    pivot = pd.pivot_table(
        stats_df, 
        values='Коэфф.вариации(%)', 
        index='Отрасль',
        columns='Строка'
    )
    
    # Создаем визуализацию
    plt.figure(figsize=(14, 10))  # Увеличил размер для вмещения новой строки
    sns.heatmap(pivot, annot=True, cmap='YlGnBu_r', fmt='.1f')
    plt.title('Коэффициенты вариации по отраслям и категориям (%)')
    plt.tight_layout()
    plt.savefig(output_file)
    plt.close()
    
    print(f"Тепловая карта стабильности сохранена в файле '{output_file}'")

def main():
    """
    Основная функция для анализа стабильности отраслей.
    """
    # Имя файла Excel
    add_part = '_by2q'
    excel_file = '3quadrant_aggr_normalized' + add_part + '.xlsx'
    
    print(f"Анализ стабильности отраслей из файла: {excel_file}")
    
    # Анализируем стабильность по всем листам Excel
    stats_df = analyze_quadrant3_stability(excel_file)
    
    if stats_df is None or stats_df.empty:
        print("Не удалось провести анализ. Проверьте файл Excel и его структуру.")
        return
    
    # Выводим общую статистику
    print("\nОбщая статистика:")
    print(f"Всего обработано пар отрасль-строка: {len(stats_df)}")
    
    # Сохраняем полную статистику в CSV с правильной кодировкой
    q3_stab_csv = 'quadrant3_stability_stats' + add_part + '.csv'
    stats_df.to_csv(q3_stab_csv, index=False, encoding='utf-8-sig')
    print(f"Полная статистика сохранена в файле {q3_stab_csv}")
    
    # Определяем наиболее стабильные отрасли
    industry_stability = identify_stable_industries(stats_df)
    
    if industry_stability is not None:
        industry_stab_csv = 'industry_stability_ranking' + add_part + '.csv'
        industry_stability.to_csv(industry_stab_csv, index=False, encoding='utf-8-sig')
        print(f"Рейтинг стабильности отраслей сохранен в файле {industry_stab_csv}")
    
    # Проводим анализ для CES-функций
    ces_df = analyze_stability_for_ces(stats_df)
    
    if ces_df is not None and not ces_df.empty:
        print("\nАнализ для CES-функций:")
        print(ces_df)
        ces_csv = 'ces_industry_metrics' + add_part + ".csv"
        ces_df.to_csv(ces_csv, index=False, encoding='utf-8-sig')
        print(f"Метрики для CES-функций сохранены в файле {ces_csv}")
    
    # Создаем тепловую карту для визуализации стабильности
    heatmap_file = 'stability_heatmap' + add_part + '.png'
    create_stability_heatmap(stats_df, heatmap_file)

if __name__ == "__main__":
    main()

Анализ стабильности отраслей из файла: 3quadrant_aggr_normalized_by2q.xlsx
Найдено листов в Excel-файле: 7
Загружены данные с листа '2017'. Размерность: (7, 6)
Загружены данные с листа '2018'. Размерность: (7, 6)
Загружены данные с листа '2019'. Размерность: (7, 6)
Загружены данные с листа '2020'. Размерность: (7, 6)
Загружены данные с листа '2021'. Размерность: (7, 6)
Загружены данные с листа '2022'. Размерность: (7, 6)
Загружены данные с листа '2023'. Размерность: (7, 6)

Общая статистика:
Всего обработано пар отрасль-строка: 42
Полная статистика сохранена в файле quadrant3_stability_stats_by2q.csv

Топ-3 наиболее стабильных отраслей:
                                             Отрасль  Средний_коэфф_вариации
4          Промышленное производство и бизнес-услуги                8.287951
1  Инвестиционные товары и капитальное строительство               10.653255
3                    Потребительские товары и услуги               11.593108
Рейтинг стабильности отраслей сохранен в файле 

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Упрощенное описание строк
ROW_NAMES = [
    "Использовано в ценах покупателя",  # 0 (было 73)
    "Оплата труда",                     # 1 (было 74)
    "Другие налоги на производство",    # 2 (было 75)
    "Потребление основного капитала",   # 3 (было 76)
    "Прибыль, смешанный доход",         # 4 (было 77)
    "Доход (Оплата труда + Прибыль)",   # 5 (Новая строка)
    "Импорт"                            # 6 (было 80)
]

def analyze_quadrant3_stability(excel_file):
    """
    Анализирует стабильность показателей 3 квадранта по листам Excel.
    """
    try:
        # Получаем список листов в файле Excel
        excel = pd.ExcelFile(excel_file)
        sheet_names = excel.sheet_names
        
        print(f"Найдено листов в Excel-файле: {len(sheet_names)}")
        
        # Создаем словарь для хранения данных по листам
        data_by_sheet = {}
        
        # Загружаем данные из каждого листа
        for sheet in sheet_names:
            try:
                # Загружаем данные из листа
                df = pd.read_excel(excel_file, sheet_name=sheet)
                
                # Создаем новый DataFrame с тем же количеством столбцов
                new_df = pd.DataFrame(index=range(len(ROW_NAMES)), columns=df.columns)
                
                # Копируем существующие строки
                for i in range(min(5, len(df))):  # Первые 5 строк
                    new_df.iloc[i] = df.iloc[i]
                
                # Вычисляем новую строку: Доход = Оплата труда + Прибыль
                if len(df) >= 5:  # Убеждаемся, что у нас есть доступ к нужным строкам
                    new_df.iloc[5] = df.iloc[1] + df.iloc[4]  # Оплата труда + Прибыль
                
                # Копируем строку Импорт
                if len(df) >= 6:
                    new_df.iloc[6] = df.iloc[5]  # Импорт
                
                # Сохраняем новые данные для листа
                data_by_sheet[sheet] = new_df
                
                print(f"Загружены данные с листа '{sheet}'. Размерность: {new_df.shape}")
            except Exception as e:
                print(f"Ошибка при обработке листа {sheet}: {e}")
        
        if not data_by_sheet:
            print("Не удалось загрузить данные ни с одного листа")
            return None
        
        # Получаем общий список отраслей (колонок)
        all_industries = set()
        for df in data_by_sheet.values():
            all_industries.update(df.columns)
        
        # Создаем список для хранения результатов
        stats_list = []
        
        # Для каждой отрасли и строки собираем данные по всем листам
        for row_idx, row_name in enumerate(ROW_NAMES):
            for industry in all_industries:
                values = []
                
                # Собираем значения для этой пары из всех листов
                for sheet, df in data_by_sheet.items():
                    if industry in df.columns and row_idx < len(df):
                        values.append(df.iloc[row_idx][industry])
                
                # Если есть хотя бы 2 значения, считаем статистику
                if len(values) >= 2:
                    mean_value = np.mean(values)
                    std_value = np.std(values)
                    
                    # Вычисляем коэффициент вариации
                    if mean_value != 0:
                        cv_value = (std_value / abs(mean_value)) * 100
                    else:
                        cv_value = np.nan
                    
                    # Добавляем результаты
                    stats_list.append({
                        'Строка': row_name,
                        'Отрасль': industry,
                        'Среднее': mean_value,
                        'Станд.откл': std_value,
                        'Коэфф.вариации(%)': cv_value,
                        'Кол-во_листов': len(values)
                    })
        
        # Преобразуем в DataFrame
        stats_df = pd.DataFrame(stats_list)
        
        return stats_df
    
    except Exception as e:
        print(f"Ошибка при анализе файла Excel: {e}")
        return None

def identify_stable_industries(stats_df, top_n=3):
    """
    Определяет наиболее стабильные отрасли.
    """
    if stats_df is None or stats_df.empty:
        return None
    
    # Группируем по отраслям и находим средний коэффициент вариации
    industry_stability = stats_df.groupby('Отрасль')['Коэфф.вариации(%)'].mean().reset_index()
    industry_stability.columns = ['Отрасль', 'Средний_коэфф_вариации']
    
    # Сортируем от наиболее стабильных к наименее
    industry_stability = industry_stability.sort_values('Средний_коэфф_вариации')
    
    print(f"\nТоп-{top_n} наиболее стабильных отраслей:")
    print(industry_stability.head(top_n))
    
    return industry_stability

def analyze_stability_for_ces(stats_df):
    """
    Проводит анализ для выбора отраслей, подходящих для CES-функций.
    """
    if stats_df is None or stats_df.empty:
        return None
    
    # Преобразуем DataFrame для упрощения анализа
    pivot_df = pd.pivot_table(
        stats_df, 
        values='Среднее', 
        index='Отрасль',
        columns='Строка'
    )
    
    # Преобразуем DataFrame для коэффициентов вариации
    cv_pivot = pd.pivot_table(
        stats_df, 
        values='Коэфф.вариации(%)', 
        index='Отрасль',
        columns='Строка'
    )
    
    # Список для хранения метрик CES
    ces_results = []
    
    for industry in pivot_df.index:
        # Получаем значения для этой отрасли
        ind_data = pivot_df.loc[industry]
        ind_cv = cv_pivot.loc[industry]
        
        # Базовые компоненты
        labor = ind_data['Оплата труда']
        capital = ind_data['Потребление основного капитала']
        imports = ind_data['Импорт']
        profit = ind_data['Прибыль, смешанный доход']
        taxes = ind_data['Другие налоги на производство']
        income = ind_data['Доход (Оплата труда + Прибыль)']
        
        # Соотношение труд/капитал
        labor_capital_ratio = labor / capital if capital != 0 else np.nan
        
        # Стабильность по коэффициентам вариации
        labor_cv = ind_cv['Оплата труда']
        capital_cv = ind_cv['Потребление основного капитала']
        import_cv = ind_cv['Импорт']
        income_cv = ind_cv['Доход (Оплата труда + Прибыль)']
        
        # Средняя стабильность соотношения труд/капитал
        labor_capital_stability = (labor_cv + capital_cv) / 2
        
        # Добавленная стоимость
        value_added = labor + taxes + capital + profit
        
        # Доли компонентов
        labor_share = labor / value_added * 100 if value_added > 0 else 0
        capital_share = capital / value_added * 100 if value_added > 0 else 0
        profit_share = profit / value_added * 100 if value_added > 0 else 0
        income_share = income / value_added * 100 if value_added > 0 else 0
        
        # Комплексный индекс стабильности (меньше = лучше)
        ces_index = (
            0.4 * labor_capital_stability +  # 40% вес
            0.3 * import_cv +                # 30% вес
            0.3 * (imports * 100)            # 30% вес
        )
        
        # Добавляем результаты
        ces_results.append({
            'Отрасль': industry,
            'Труд/Капитал': labor_capital_ratio,
            'Стабильность_ТК': labor_capital_stability,
            'Доля_импорта': imports * 100,
            'Стабильность_импорта': import_cv,
            'Доля_труда': labor_share,
            'Доля_капитала': capital_share,
            'Доля_прибыли': profit_share,
            'Доля_дохода': income_share,
            'Стабильность_дохода': income_cv,
            'CES_индекс': ces_index
        })
    
    # Создаем DataFrame и сортируем по индексу стабильности
    ces_df = pd.DataFrame(ces_results)
    if not ces_df.empty:
        ces_df = ces_df.sort_values('CES_индекс')
    
    return ces_df

def create_stability_heatmap(stats_df, output_file='stability_heatmap.png'):
    """
    Создает тепловую карту коэффициентов вариации.
    """
    if stats_df is None or stats_df.empty:
        print("Нет данных для создания тепловой карты")
        return
    
    # Создаем сводную таблицу для тепловой карты
    pivot = pd.pivot_table(
        stats_df, 
        values='Коэфф.вариации(%)', 
        index='Отрасль',
        columns='Строка'
    )
    
    # Создаем визуализацию
    plt.figure(figsize=(14, 10))  # Увеличил размер для вмещения новой строки
    sns.heatmap(pivot, annot=True, cmap='YlGnBu_r', fmt='.1f')
    plt.title('Коэффициенты вариации по отраслям и категориям (%)')
    plt.tight_layout()
    plt.savefig(output_file)
    plt.close()
    
    print(f"Тепловая карта стабильности сохранена в файле '{output_file}'")

def main():
    """
    Основная функция для анализа стабильности отраслей.
    """
    # Имя файла Excel
    add_part = '_byLabor'
    excel_file = '3quadrant_aggr_normalized' + add_part + '.xlsx'
    
    print(f"Анализ стабильности отраслей из файла: {excel_file}")
    
    # Анализируем стабильность по всем листам Excel
    stats_df = analyze_quadrant3_stability(excel_file)
    
    if stats_df is None or stats_df.empty:
        print("Не удалось провести анализ. Проверьте файл Excel и его структуру.")
        return
    
    # Выводим общую статистику
    print("\nОбщая статистика:")
    print(f"Всего обработано пар отрасль-строка: {len(stats_df)}")
    
    # Сохраняем полную статистику в CSV с правильной кодировкой
    q3_stab_csv = 'quadrant3_stability_stats' + add_part + '.csv'
    stats_df.to_csv(q3_stab_csv, index=False, encoding='utf-8-sig')
    print(f"Полная статистика сохранена в файле {q3_stab_csv}")
    
    # Определяем наиболее стабильные отрасли
    industry_stability = identify_stable_industries(stats_df)
    
    if industry_stability is not None:
        industry_stab_csv = 'industry_stability_ranking' + add_part + '.csv'
        industry_stability.to_csv(industry_stab_csv, index=False, encoding='utf-8-sig')
        print(f"Рейтинг стабильности отраслей сохранен в файле {industry_stab_csv}")
    
    # Проводим анализ для CES-функций
    ces_df = analyze_stability_for_ces(stats_df)
    
    if ces_df is not None and not ces_df.empty:
        print("\nАнализ для CES-функций:")
        print(ces_df)
        ces_csv = 'ces_industry_metrics' + add_part + ".csv"
        ces_df.to_csv(ces_csv, index=False, encoding='utf-8-sig')
        print(f"Метрики для CES-функций сохранены в файле {ces_csv}")
    
    # Создаем тепловую карту для визуализации стабильности
    heatmap_file = 'stability_heatmap' + add_part + '.png'
    create_stability_heatmap(stats_df, heatmap_file)

if __name__ == "__main__":
    main()

Анализ стабильности отраслей из файла: 3quadrant_aggr_normalized_byLabor.xlsx
Найдено листов в Excel-файле: 7
Загружены данные с листа '2017'. Размерность: (7, 19)
Загружены данные с листа '2018'. Размерность: (7, 19)
Загружены данные с листа '2019'. Размерность: (7, 19)
Загружены данные с листа '2020'. Размерность: (7, 19)
Загружены данные с листа '2021'. Размерность: (7, 19)
Загружены данные с листа '2022'. Размерность: (7, 19)
Загружены данные с листа '2023'. Размерность: (7, 19)

Общая статистика:
Всего обработано пар отрасль-строка: 133
Полная статистика сохранена в файле quadrant3_stability_stats_byLabor.csv

Топ-3 наиболее стабильных отраслей:
                                             Отрасль  Средний_коэфф_вариации
2  Государственное управление и оборона; обязател...                1.488912
7                      Обрабатывающая промышленность                3.103970
5                                 Информация и связь                4.345771
Рейтинг стабильности отраслей сох

KeyError: 'Потребление основного капитала'